# NOTEBOOK COPIA EJEMPLO NIXTLA hierarchicalforecast

----
### ejemplo conciliacion GEOGRÁFICA (POINT RECONCILIATION) de los datos
- Ejecución de notebook ejemplo base de nixtla - replicar código ejemplo y análisis extra para entender código y modelos ya hechos

- EJEMPLO LA DATA FULL DE TOURISM

- FUENTES:
    - geographical aggregation (POINT CONCILIATION): https://nixtlaverse.nixtla.io/hierarchicalforecast/examples/australiandomestictourism.html

    - temporal aggregation (TEMPORAL CONCILIATION): https://nixtlaverse.nixtla.io/hierarchicalforecast/examples/australiandomestictourismtemporal.html
 
    - geographical and temporal aggregation: https://nixtlaverse.nixtla.io/hierarchicalforecast/examples/australiandomestictourismcrosstemporal.html
 
    - paper base investigación (los códigos de nixtla replican lo obtenido por el paper): https://robjhyndman.com/seminars/fr_overview.html

# POINT RECONCILIATION: Geographical Aggregation (Tourism)
FUENTE BASE: https://nixtlaverse.nixtla.io/hierarchicalforecast/examples/australiandomestictourism.html

### 0. Install nixtla package

In [1]:
# pip install hierarchicalforecast
# pip install datasetsforecast
# pip install statsforecast

In [2]:
import hierarchicalforecast
from datasetsforecast.hierarchical import HierarchicalData
import pandas as pd

## run code - example nixtla

In [3]:
import numpy as np
import pandas as pd

### 1. Read data - download data example hyndman

#### 1.1 read raw data

In [4]:
data_raw = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
data_raw = data_raw.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
data_raw.insert(0, 'Country', 'Australia')
data_raw = data_raw[['Country', 'Region', 'State', 'Purpose', 'ds', 'y']]
data_raw['ds'] = data_raw['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
data_raw['ds'] = pd.PeriodIndex(data_raw["ds"], freq='Q').to_timestamp()

In [5]:
# print data - se observa que a diferencia del ejemplo small que se carga la matriz S para realizar la reconciliación de forecast
# aquí estan todas las series y hay que armar la matriz de cómo se relacionan los forecasts
data_raw.head()

,Country,Region,State,Purpose,ds,y
0,Australia,Adelaide,South Australia,Business,1998-01-01,135.077690
1,Australia,Adelaide,South Australia,Business,1998-04-01,109.987316
2,Australia,Adelaide,South Australia,Business,1998-07-01,166.034687
3,Australia,Adelaide,South Australia,Business,1998-10-01,127.160464
4,Australia,Adelaide,South Australia,Business,1999-01-01,137.448533


In [6]:
# OBS: NOTAR ADEMÁS QUE AQUÍ ESTÁN TODAS LAS SERIES AL NIVEL MÁS DESAGREGADO POSIBLE 
# (EN LOS SIGUIENTES PASOS SE GENERA LA DATA CON MAYOR NIVEL DE AGREGACIÓN, EJ A NIVEL COUNTRY)

#### 1.2 Cargar agregación de los datos - cómo se agrupan las series en los diferentes niveles

In [7]:
# generar lista/matriz de python con las columnas del dataframe que corresponde al "unique_id" todos los diferentes grados de desagregación
# que pueden tener los datos
# EN ESTE EJEMPLO DESAGREGACIÓN GEOGRÁFICA DE LOS DATOS (no considera desagregaciones temporales)
spec = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]

#### 1.3 Generar el conjunto completo de series y las matrices para realizar la consolidación de los resultados (S, tags)
**nixtla tiene la función para generar la agregación**

In [8]:
from hierarchicalforecast.utils import aggregate

In [9]:
Y_df, S_df, tags = aggregate(data_raw, spec)

In [10]:
# print head diferentes dataframes generados en la agregación de las series

In [11]:
Y_df.head(3)

,unique_id,ds,y
0,Australia,1998-01-01,23182.197269
1,Australia,1998-04-01,20323.380067
2,Australia,1998-07-01,19826.640511


In [12]:
Y_df.tail(3)

,unique_id,ds,y
33997,Australia/Western Australia/Experience Perth/V...,2017-04-01,302.296119
33998,Australia/Western Australia/Experience Perth/V...,2017-07-01,373.442070
33999,Australia/Western Australia/Experience Perth/V...,2017-10-01,455.316702


In [13]:
# ----->OJO
# SE PUEDE OBSERVAR QUE AL CREAR LAS NUEVAS SERIES CON LA AGREGACIÓN GEOGRÁFICA (POINT AGREGATION), LAS NUEVAS SERIES CREADAS
# SE GUARDAN EN LA MISMA COLUMNA "UNIQUE_ID"

# EN EL NOTEBOOK DE AGREGACIÓN TEMPORAL SE PUEDE VER AL CREAR LAS NUEVAS SERIES TEMPORALES AGREGAS SE CREA UNA NUEVA COLUMNA "TEMPORAL_ID"
# PARA INDENTIFICAR LAS NUEVAS SERIES AGREGADAS TEMPORALMENTE

In [14]:
S_df.iloc[:5, :5]

,unique_id,Australia/ACT/Canberra/Business,Australia/ACT/Canberra/Holiday,Australia/ACT/Canberra/Other,Australia/ACT/Canberra/Visiting
0,Australia,1.0,1.0,1.0,1.0
1,Australia/ACT,1.0,1.0,1.0,1.0
2,Australia/New South Wales,0.0,0.0,0.0,0.0
3,Australia/Northern Territory,0.0,0.0,0.0,0.0
4,Australia/Queensland,0.0,0.0,0.0,0.0


In [15]:
tags['Country/Purpose']

array(['Australia/Business', 'Australia/Holiday', 'Australia/Other',
       'Australia/Visiting'], dtype=object)

In [16]:
# shape data raw
data_raw.shape

(24320, 6)

In [17]:
# shape y_df - se generan más series
Y_df.shape

(34000, 3)

In [18]:
# matriz matriz S
S_df.shape

(425, 305)

In [19]:
tags.keys()

dict_keys(['Country', 'Country/State', 'Country/Purpose', 'Country/State/Region', 'Country/State/Purpose', 'Country/State/Region/Purpose'])

### 2. Discovery Data (fuente origen propia)

#### 2.1 discovery data_raw

In [20]:
# generar columna unique_id
data_raw['unique_id'] = data_raw['Country'] + '/' + data_raw['State'] + '/' + data_raw['Region'] + '/' + data_raw['Purpose']
data_raw.head(3)

,Country,Region,State,Purpose,ds,y,unique_id
0,Australia,Adelaide,South Australia,Business,1998-01-01,135.077690,Australia/South Australia/Adelaide/Business
1,Australia,Adelaide,South Australia,Business,1998-04-01,109.987316,Australia/South Australia/Adelaide/Business
2,Australia,Adelaide,South Australia,Business,1998-07-01,166.034687,Australia/South Australia/Adelaide/Business


In [21]:
# print tamaño data RAW
data_raw.shape

(24320, 7)

In [22]:
# print algunos ejemplo de cada serie
unique_values_country = data_raw['Country'].unique().tolist()
unique_values_state = data_raw['State'].unique().tolist()
unique_values_region = data_raw['Region'].unique().tolist()
unique_values_purpose = data_raw['Purpose'].unique().tolist()
print('\nunique_values_country: ', unique_values_country)
print('\nunique_values_state: ', unique_values_state)
print('\nunique_values_region (example 10): ', unique_values_region[0:10])
print('\nunique_values_purpose: ', unique_values_purpose)


unique_values_country:  ['Australia']

unique_values_state:  ['South Australia', 'Northern Territory', 'Western Australia', 'Victoria', 'New South Wales', 'Queensland', 'ACT', 'Tasmania']

unique_values_region (example 10):  ['Adelaide', 'Adelaide Hills', 'Alice Springs', "Australia's Coral Coast", "Australia's Golden Outback", "Australia's North West", "Australia's South West", 'Ballarat', 'Barkly', 'Barossa']

unique_values_purpose:  ['Business', 'Holiday', 'Other', 'Visiting']


In [23]:
# combinaciones únicas de cada columna
print('shape_unique_values_country: ', len(unique_values_country))
print('shape_unique_values_state: ', len(unique_values_state))
print('shape_unique_values_region: ', len(unique_values_region))
print('shape_unique_values_purpose: ', len(unique_values_purpose))

shape_unique_values_country:  1
shape_unique_values_state:  8
shape_unique_values_region:  76
shape_unique_values_purpose:  4


In [24]:
# cantidad de series únicas en los datos
data_raw['unique_id'].unique().shape

(304,)

In [25]:
# se ve que no todas las combinaciones existen como series
len(unique_values_country) * len(unique_values_state) * len(unique_values_region) * len(unique_values_purpose)

2432

In [26]:
# print fecha de inicio y fin de los datos (OJO TODAS LAS SERIES TIENEN LA MISMA CANTIDAD DE DATOS y las mismas fechas de inicio y fin)
example_unique_id = data_raw['unique_id'][0]
fecha_inicio_raw = data_raw[data_raw['unique_id'] == example_unique_id]['ds'].min()
fecha_fin_raw = data_raw[data_raw['unique_id'] == example_unique_id]['ds'].max()
shape_unique_serie = data_raw[data_raw['unique_id'] == example_unique_id].shape

print('fecha inicio data: ', fecha_inicio_raw)
print('fecha fin data: ', fecha_fin_raw)
print('shape serie individual: ', shape_unique_serie)

fecha inicio data:  1998-01-01 00:00:00
fecha fin data:  2017-10-01 00:00:00
shape serie individual:  (80, 7)


In [27]:
# cantidad de series por tamaño de serie, igual al tamaño del dataframe (validar que todas las series tienen la misma cantidad de datos)
(80 * 304) == data_raw.shape[0]

True

#### 2.2 discovery data_agregada (data luego de aplicar función de agregación)

In [28]:
# print tamaño data
Y_df.shape

(34000, 3)

In [29]:
# cantidad de series únicas en los datos
Y_df['unique_id'].unique().shape

(425,)

#### 2.3 Validar qué series están en la data raw y cuales no

In [30]:
# cantidad de series cada data
list_unique_id_data_raw = data_raw['unique_id'].unique().tolist()
list_unique_id_data_ydf = Y_df['unique_id'].unique().tolist()
print('list_unique_id_data_raw: ', len(list_unique_id_data_raw))
print('list_unique_id_data_ydf: ',len(list_unique_id_data_ydf))

list_unique_id_data_raw:  304
list_unique_id_data_ydf:  425


In [31]:
# qué series están en cada data

In [32]:
# cantidad de series que están en la data raw y no en la data agregada (DEBERÍA ESTAR TODA LA DATA)
len(list(set(list_unique_id_data_raw) - set(list_unique_id_data_ydf)))

0

In [33]:
# cantidad de series que están en la data agregada y NO están en la raw (DEBERÍAN ESTAR TODAS LAS SERIES AGREGADAS)
len(list(set(list_unique_id_data_ydf) - set(list_unique_id_data_raw)))

121

In [34]:
# validar que restar la data agregada menos la data raw, se obtiene la diferencia
len(list_unique_id_data_ydf) - len(list_unique_id_data_raw)

121

**---> ENTONCES LA DATA RAW QUE SE OBTUVO ES LA MÁS DESAGREGADA - LAS SERIES AL NIVEL MÁS INDIVIDUAL**

### 3. Split Train/Test sets (continuación código base nixtla)
We use the final two years (8 quarters) as test set

In [35]:
# param - horizonte a forecastear
horizon = 8

In [36]:
Y_test_df = Y_df.groupby('unique_id', as_index=False).tail(horizon)
Y_train_df = Y_df.drop(Y_test_df.index)

In [37]:
# print informativo tamaño de la data original, data_train y data_test
print('Y_df: ', Y_df.shape)
print('Y_train_df: ', Y_train_df.shape)
print('Y_test_df: ', Y_test_df.shape)

Y_df:  (34000, 3)
Y_train_df:  (30600, 3)
Y_test_df:  (3400, 3)


### 4. Computing base forecasts

In [38]:
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast

In [39]:
fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], 
                     freq='QS', n_jobs=-1)

Y_hat_df = fcst.forecast(df=Y_train_df, h=8, fitted=True)

Y_fitted_df = fcst.forecast_fitted_values()

In [40]:
# print dataframes generados

In [41]:
# TRAIN: shape
Y_train_df.shape

(30600, 3)

In [42]:
# TRAIN: print fecha min y max (ojo, recordar que todas las series tienen la misma fecha de inicio y fin)
min_fecha_train = Y_train_df['ds'].min()
max_fecha_train = Y_train_df['ds'].max()
print('min_fecha_train: ', min_fecha_train)
print('max_fecha_train: ', max_fecha_train)

min_fecha_train:  1998-01-01 00:00:00
max_fecha_train:  2015-10-01 00:00:00


In [43]:
# Y_HAT: shape (debería el dataframe con los datos forecasteados)
Y_hat_df.shape

(3400, 3)

In [44]:
Y_hat_df['unique_id'].unique().shape[0] * 8 # multiplicar cantidad de series por horizonte (efectivamente y_hat_df corresponde al df de forecasts)

3400

In [45]:
# Y_HAT: fecha min y máximo
min_fecha_future_forecast = Y_hat_df['ds'].min()
max_fecha_future_forecast = Y_hat_df['ds'].max()
print('min_fecha_future_forecast: ', min_fecha_future_forecast)
print('max_fecha_future_forecast: ', max_fecha_future_forecast)

min_fecha_future_forecast:  2016-01-01 00:00:00
max_fecha_future_forecast:  2017-10-01 00:00:00


In [46]:
# Y_fitted_df: shape (debería ser el dataframe que tiene el forecast para los datos de train y además el forecast test/future) FALSO
# Y_fitted_df: shape (ES EL DATAFRAME DE TRAIN QUE SE LE AGREGA EL FORECAST GENERADO EN TRAIN)
Y_fitted_df.shape

(30600, 4)

In [47]:
# mostrar tamaño de TRAIN y validar que SI coincide con Y_fitted_df
Y_train_df.shape

(30600, 3)

In [48]:
# Y_fitted_df: print fecha min y max
min_fecha_train_fcst = Y_fitted_df['ds'].min()
max_fecha_train_fcst = Y_fitted_df['ds'].max()
print('min_fecha_train_fcst: ', min_fecha_train_fcst)
print('max_fecha_train_fcst: ', max_fecha_train_fcst)

min_fecha_train_fcst:  1998-01-01 00:00:00
max_fecha_train_fcst:  2015-10-01 00:00:00


### 5. Validar que los forecast no son coherentes (al hacer la agregacion geográfica el volumen no coincide)

In [49]:
# consultar el diccionario de tags para saber los diferentes niveles de agregacion
tags.keys()

dict_keys(['Country', 'Country/State', 'Country/Purpose', 'Country/State/Region', 'Country/State/Purpose', 'Country/State/Region/Purpose'])

In [50]:
# obtener las series de "Country"
list_series_country = tags['Country'].tolist()
list_series_country

['Australia']

In [51]:
# obtener las series de "Country/State" (los diferentes estados que conforman el país) (si se sumara el forecast de cada estado debería dar el del país)
list_series_country_state = tags['Country/State'].tolist()
list_series_country_state

['Australia/ACT',
 'Australia/New South Wales',
 'Australia/Northern Territory',
 'Australia/Queensland',
 'Australia/South Australia',
 'Australia/Tasmania',
 'Australia/Victoria',
 'Australia/Western Australia']

In [52]:
# filtrar forecast país para una fecha en específico (para comparar vs la suma del forecast cada state por separado y luego sumándolo)
fecha_example_filter = '2016-01-01'

mask_example_country = (Y_hat_df['unique_id'].isin(list_series_country)) & (Y_hat_df['ds'] == fecha_example_filter)
Y_hat_df[mask_example_country]

,unique_id,ds,AutoETS
0,Australia,2016-01-01,25990.068004


In [53]:
# filtrar forecast por cada state para una fecha en específico, luego sumar y comparar si coincide con forecast a nivel país
fecha_example_filter = '2016-01-01'

mask_example_country_state = (Y_hat_df['unique_id'].isin(list_series_country_state)) & (Y_hat_df['ds'] == fecha_example_filter)
Y_hat_df[mask_example_country_state]

,unique_id,ds,AutoETS
8,Australia/ACT,2016-01-01,553.037571
104,Australia/New South Wales,2016-01-01,7950.122623
664,Australia/Northern Territory,2016-01-01,290.492099
992,Australia/Queensland,2016-01-01,5179.108984
1512,Australia/South Australia,2016-01-01,1715.447017
2032,Australia/Tasmania,2016-01-01,914.333862
2272,Australia/Victoria,2016-01-01,6301.092079
3160,Australia/Western Australia,2016-01-01,2732.699242


In [54]:
# sumar forecast por cada state (SE PUEDE VER QUE LA SUMA DE FORECAST DA UN VOLUMEN DISTINTO VS FORECAST A NIVEL PAÍS)
Y_hat_df[mask_example_country_state]['AutoETS'].sum()

25636.33347680661

### 6. RECONCILE FORECAST!!!
The following cell makes the previous forecasts coherent using the HierarchicalReconciliation class. Since the hierarchy structure is not strict, we can’t use methods such as TopDown or MiddleOut. In this example we use BottomUp and MinTrace.

In [55]:
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

In [56]:
# cargar listado de posibles formas de reconciliar forecasts (bottomup y MinTrace)
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]

# llamar instancia de objeto con los "modelos" para conciliar forecasts
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
hrec

In [57]:
# ---- OJO PARA RECONCILIAR FORECASTS SE NECESITA: ----
# Y_hat_df: dataframe con los datos FUTURE. datos forecasteados
# Y_fitted_df: dataframe con los datos de train y el forecast generado (sería el equivalente a y_train, y_train_pred) 
# S_df: matriz que se genera con nixtla (matriz binaria donde se muestra qué forecasts están relacionados)
# tags: diccionario que se genera con nixtla (tiene las individuales y cómo se relacionan para generar las agregaciones)

# (al final se ajusta el error (conciliación min_trace) en train con el real y el predicho y luego se aplica esa conciliación al forecast generado)

In [58]:
# ---- OJO IMPORTANTE 2
# para la reconciliación "mint_shrink" se necesita pasar los datos Y_df = Y_fitted_df (datos de train con real y predicho)
# pero para las otras reconciliaciones solo se necesita pasar los forecast generados (NO SE NECESITA TRAIN)

# ejemplo
# reconcilers = [
#     BottomUp(),
#     MinTrace(method='ols')
# ]

# hrec = HierarchicalReconciliation(reconcilers=reconcilers)

# hrec.reconcile(Y_hat_df = Y_hat_df,
#                           S = S_df, 
#                           tags = tags) # ojo no se pasa Y_df = Y_fitted_df, ya que no se usa la conciliacion mint_shrink

In [59]:
# reconciliar forecasts
Y_rec_df = hrec.reconcile(Y_hat_df = Y_hat_df, 
                          Y_df = Y_fitted_df, 
                          S = S_df, 
                          tags = tags)

In [60]:
# print dataframe con las reconciliaciones
Y_rec_df.head(3)

,unique_id,ds,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,24381.672902,25427.793552,25894.419896
1,Australia,2016-04-01,24458.490282,22903.194015,23913.800914,24357.231461
2,Australia,2016-07-01,23974.055984,22411.401316,23428.540858,23865.928094


### 7. VALIDAR QUE SE CONSERVA EL VOLUMEN CON LAS RECONCILIACIONES
Volver a hacer print de una fecha de ejemplo (y una agregación de ejemplo) y ahora validar que se mantiene el volumen
UTILIZAR EL DATAFRAME NUEVO CON LOS FORECAST RECONCILIADOS "Y_rec_df"

In [61]:
# obtener las series de "Country"
list_series_country = tags['Country'].tolist()
list_series_country

['Australia']

In [62]:
# obtener las series de "Country/State" (los diferentes estados que conforman el país) (si se sumara el forecast de cada estado debería dar el del país)
list_series_country_state = tags['Country/State'].tolist()
list_series_country_state

['Australia/ACT',
 'Australia/New South Wales',
 'Australia/Northern Territory',
 'Australia/Queensland',
 'Australia/South Australia',
 'Australia/Tasmania',
 'Australia/Victoria',
 'Australia/Western Australia']

In [63]:
# obtener las series 'Country/State/Region/Purpose' (EL NIVEL DE MAYOR DESAGREGACIÓN DEL FORECAST)
list_series_all_desagregacion = tags['Country/State/Region/Purpose'].tolist()
len(list_series_all_desagregacion)

304

In [64]:
# filtrar forecast país para una fecha en específico (para comparar vs la suma del forecast cada state por separado y luego sumándolo)
fecha_example_filter = '2016-01-01'

mask_example_country = (Y_rec_df['unique_id'].isin(list_series_country)) & (Y_rec_df['ds'] == fecha_example_filter)
Y_rec_df[mask_example_country]

,unique_id,ds,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,24381.672902,25427.793552,25894.419896


In [65]:
# filtrar forecast por cada state para una fecha en específico, luego sumar y comparar si coincide con forecast a nivel país
fecha_example_filter = '2016-01-01'

mask_example_country_state = (Y_rec_df['unique_id'].isin(list_series_country_state)) & (Y_rec_df['ds'] == fecha_example_filter)
Y_rec_df_example = Y_rec_df[mask_example_country_state]


cols_to_sum = ['AutoETS', 'AutoETS/BottomUp', 'AutoETS/MinTrace_method-mint_shrink', 'AutoETS/MinTrace_method-ols']
Y_rec_df_example[cols_to_sum].sum(axis=0).to_frame(name='sum_result').T

,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
sum_result,25636.333477,24381.672902,25427.793552,25894.419896


In [66]:
# ojo BottomUp probablemente sumó desde lo más abajo.
# FILTRAR FORECAST DEL NIVEL DE MAYOR DESEGREGACION Y SUMAR HASTA LLEGAR A COUNTRY. validar que bottom/up sumó desde el nivel de mayor desagregacion
fecha_example_filter = '2016-01-01'

mask_example_all_desagregation = (Y_rec_df['unique_id'].isin(list_series_all_desagregacion)) & (Y_rec_df['ds'] == fecha_example_filter)
Y_rec_df_example = Y_rec_df[mask_example_all_desagregation]

cols_to_sum = ['AutoETS', 'AutoETS/BottomUp', 'AutoETS/MinTrace_method-mint_shrink', 'AutoETS/MinTrace_method-ols']
Y_rec_df_example[cols_to_sum].sum(axis=0).to_frame(name='sum_result').T

,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
sum_result,24381.672902,24381.672902,25427.793552,25894.419896


In [67]:
# SE PUEDE VER QUE PARA EL MAYOR NIVEL DE DESAGREGACION sumar los forecast individuales para llegar a country el total fue de 24381.672902
# LUEGO AL OBSERVAR LA CONCILIACION BOTTOM/UP el forecast conciliado a nivel country fue efectivamente de 24381.672902
# ASI LA CONCILACIÓN BOTTOM/UP TOMA DESDE EL MAYOR GRADO DE DESAGREGACIÓN Y COMIENZA A SUMAR AGUAS ARRIBA

## 8. Evaluation
The HierarchicalForecast package includes an evaluate function to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [68]:
from hierarchicalforecast.evaluation import evaluate
from utilsforecast.losses import rmse, mase
from functools import partial

In [69]:
#### crear diccionario con los diferentes nivel de desagregación de los datos (practicamente es el diccionatio TAGS solo que se cambian el nombre de las keys)
eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['Purpose'] = tags['Country/Purpose']
eval_tags['State'] = tags['Country/State']
eval_tags['Regions'] = tags['Country/State/Region']
eval_tags['Bottom'] = tags['Country/State/Region/Purpose']

In [70]:
eval_tags.keys()

dict_keys(['Total', 'Purpose', 'State', 'Regions', 'Bottom'])

In [71]:
tags.keys()

dict_keys(['Country', 'Country/State', 'Country/Purpose', 'Country/State/Region', 'Country/State/Purpose', 'Country/State/Region/Purpose'])

In [72]:
#### generar df con los reales y los forecasts

In [73]:
# print data real
Y_test_df.head(3)

,unique_id,ds,y
72,Australia,2016-01-01,26660.637689
73,Australia,2016-04-01,24285.027757
74,Australia,2016-07-01,24191.320131


In [74]:
# print data con los forecast (diferentes conciliaciones)
Y_rec_df.head(3)

,unique_id,ds,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,24381.672902,25427.793552,25894.419896
1,Australia,2016-04-01,24458.490282,22903.194015,23913.800914,24357.231461
2,Australia,2016-07-01,23974.055984,22411.401316,23428.540858,23865.928094


In [75]:
# generar dataframe con el real y los forecasts
df = Y_rec_df.merge(Y_test_df, on=['unique_id', 'ds'])

In [76]:
df.head(3)

,unique_id,ds,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols,y
0,Australia,2016-01-01,25990.068004,24381.672902,25427.793552,25894.419896,26660.637689
1,Australia,2016-04-01,24458.490282,22903.194015,23913.800914,24357.231461,24285.027757
2,Australia,2016-07-01,23974.055984,22411.401316,23428.540858,23865.928094,24191.320131


In [77]:
##### llamar funcion nixtla para evaluar forecasts
evaluation = evaluate(df = df, # df con real y forecasts
                      tags = eval_tags, # diccionario con las diferentes agregaciones (la verdad no entiendo para qué lo necesita)
                      train_df = Y_train_df, # data train (tampoco entiendo para qué los nececesita)
                      metrics = [rmse, partial(mase, seasonality=4)])


# correguir tipo de dato y presentación de los resultados
evaluation.columns = ['level', 'metric', 'Base', 'BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']
numeric_cols = evaluation.select_dtypes(include="number").columns
evaluation[numeric_cols] = evaluation[numeric_cols].map('{:.2f}'.format).astype(np.float64)

In [78]:
evaluation

,level,metric,Base,BottomUp,MinTrace(mint_shrink),MinTrace(ols)
0,Total,rmse,1743.29,3029.02,2112.94,1818.94
1,Total,mase,1.59,3.16,2.06,1.67
2,Purpose,rmse,534.75,791.28,577.18,515.53
3,Purpose,mase,1.32,2.28,1.48,1.25
4,State,rmse,308.15,413.44,316.85,287.34
5,State,mase,1.39,1.90,1.40,1.25
6,Regions,rmse,51.66,55.13,46.55,46.29
7,Regions,mase,1.12,1.19,1.01,0.99
8,Bottom,rmse,19.37,19.37,17.80,18.19
9,Bottom,mase,0.98,0.98,0.94,1.01


In [79]:
# OBSERVACIONES
# hacer conciliación bottomUp hace que se empeore mucho las métricas en total(country)
# En todos los diferentes grados de desagregación de los datos funciona mejor MinTrace(ols). Excepto en el total